## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-01-17-05-44-12 +0000,nypost,CIA Director John Ratcliffe meets with Venezue...,https://nypost.com/2026/01/17/us-news/cia-dire...
1,2026-01-17-05-36-33 +0000,startribune,Takeaways: Timberwolves’ sloppiness costly in ...,https://www.startribune.com/takeaways-timberwo...
2,2026-01-17-05-24-01 +0000,nypost,Feds in Minneapolis immigration operation can’...,https://nypost.com/2026/01/17/us-news/feds-in-...
3,2026-01-17-05-19-54 +0000,nyt,Trump Backs Down on Insurrection Act as Democr...,https://www.nytimes.com/2026/01/16/us/minnesot...
4,2026-01-17-05-01-14 +0000,nyt,The $2 Rental Batteries Helping to Power South...,https://www.nytimes.com/2026/01/17/world/afric...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2368/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
28,trump,67
97,ice,22
261,iran,20
384,new,15
56,president,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
135,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...,132
267,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...,128
88,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...,125
13,2026-01-17-04-00-00 +0000,wsj,"Entering the second year of his second term, P...",https://www.wsj.com/politics/policy/over-two-w...,124
143,2026-01-16-20-34-00 +0000,wsj,What has Trump done as president so far? The W...,https://www.wsj.com/politics/policy/donald-tru...,119


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
135,132,2026-01-16-20-48-00 +0000,wsj,President Trump will issue a plan next week fo...,https://www.wsj.com/politics/policy/trump-plan...
51,74,2026-01-17-00-59-04 +0000,cbc,U.S. Justice Department probing Minnesota Gov....,https://www.cbc.ca/news/world/tim-walz-jacob-f...
267,61,2026-01-16-12-48-22 +0000,nypost,Venezuelan opposition leader María Corina Mach...,https://nypost.com/2026/01/16/us-news/venezuel...
285,49,2026-01-16-11-42-07 +0000,bbc,US accuses South Africa military of 'cosying u...,https://www.bbc.com/news/articles/c8x95nk72k2o...
123,47,2026-01-16-21-24-32 +0000,cbc,Here's what you need to know about the Canada-...,https://www.cbc.ca/news/politics/what-is-in-ca...
323,44,2026-01-16-09-00-00 +0000,cbc,Federal government loses Emergencies Act appea...,https://www.cbc.ca/news/politics/convoy-protes...
88,43,2026-01-16-23-08-00 +0000,wsj,President Trump signaled that he might keep Na...,https://www.wsj.com/economy/central-banking/tr...
67,39,2026-01-17-00-09-17 +0000,nypost,Tyler Robinson hearing takes bizarre turn as j...,https://nypost.com/2026/01/16/us-news/tyler-ro...
120,38,2026-01-16-21-27-56 +0000,bbc,Trump threatens new tariffs on countries oppos...,https://www.bbc.com/news/articles/c9qpy952xvno...
89,37,2026-01-16-22-59-26 +0000,latimes,"In San Francisco, Newsom rails against propose...",https://www.latimes.com/california/story/2026-...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
